In [22]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Load images

In [23]:
close_paths = !ls close

In [25]:
close_images = []
for path in close_paths:
    image = tf.keras.preprocessing.image.load_img(f'close/{path}', color_mode='grayscale', target_size=(round(0.8*260), round(0.8*400)), interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    close_images.append(input_arr)

In [26]:
open_paths = !ls open/train + !ls open/test

In [27]:
open_paths = [path for path in open_paths if '.jpg' in path]

In [28]:
open_images = []
for path in open_paths:
    try:
        image = tf.keras.preprocessing.image.load_img(f'open/test/{path}', color_mode='grayscale', target_size=(round(0.8*260, 0.8*400), interpolation='nearest')
    except:
        image = tf.keras.preprocessing.image.load_img(f'open/train/{path}', color_mode='grayscale', target_size=(0.8*260, 0.8*400), interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    open_images.append(input_arr)

FileNotFoundError: [Errno 2] No such file or directory: 'open/train/openEye0.jpg'

In [19]:
close_images = np.array(close_images)

In [20]:
close_images.shape

(0,)

In [ ]:
open_images = np.array(open_images)

In [ ]:
open_images.shape

# Create labels

In [4]:
y = np.zeros((561+346))

In [5]:
y[:346] = 1

In [6]:
y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

## Create new images

In [7]:
X = np.concatenate((open_images, close_images), axis=0)

NameError: name 'open_images' is not defined

In [8]:
X_flipped_h = np.fliplr(X)

NameError: name 'X' is not defined

In [16]:
X_flipped_v = np.flipud(X)

In [17]:
X_aug = np.concatenate((X, X_flipped_h, X_flipped_v), axis=0)

## Reshape features and categorize labels

In [18]:
X = X.reshape(X.shape[0], -1)/255

In [19]:
X_aug = X_aug.reshape((X_aug.shape[0], -1))/255

In [20]:
y_aug = np.concatenate((y,y,y), axis=0)

In [21]:
y = tf.keras.utils.to_categorical(y, num_classes=2)

In [22]:
y_aug = tf.keras.utils.to_categorical(y_aug, num_classes=2)

In [23]:
X_aug.shape

(2721, 104000)

In [24]:
y_aug.shape

(2721, 2)

## Classification

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_aug, y_aug, test_size=.2, stratify=y_aug, random_state=42)

In [27]:
def mlp_model(input_dim):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(100, input_dim=input_dim, activation='relu'))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='sigmoid'))
              
    return model

In [28]:
model = mlp_model(104000)

In [29]:
from tensorflow.keras.callbacks import EarlyStopping
callback = [EarlyStopping(monitor='val_loss', patience=30)]

In [30]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

In [31]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics='accuracy')

In [ ]:
model.fit(X_train, y_train, validation_split=.15, batch_size=20, epochs=1000, callbacks=callback)